# 天池新闻推荐入门赛(新闻推荐)  Task1

## 1. 赛题理解的意义

- 理解赛题：从直观上对问题进行梳理，分析问题的目标
- 理解数据：对赛题数据做初步了解，知道和任务相关的数据以及数据之间的关联，方便之后的数据分析和特征工程。
- 理解评估指标：评估指标是检验我们提出的方法，只有正确地理解评估指标才能更好的进行模型训练和数据预测。由于线上验证有一定的次数和时间限制，所以在线下构建一个合理的本地验证集合评价指标是非常重要的。

有了赛题理解后，我们要有一些相应地理解分析：
比如难点可能在哪里，关键点可能在哪里，哪些地方可以挖掘更好的特征，用什么样的线下验证更为稳定，出现了过拟合或者其他问题，可以用哪些方法去解决这些问题。

## 2. 赛题简介

此次比赛是新闻推荐场景下的用户行为预测挑战赛。赛题以新闻APP中的新闻推荐为背景，要求我们根据用户历史浏览点击新闻文章的数据信息来预测用户未来的点击行为，即用户的最后一次点击的新闻文章。

## 3. 数据概况

数据来自某新闻APP平台的用户交互数据，包括30万用户，近300万次点击，共36万多篇不同的新闻文章，同时每篇新闻文章有对应的embedding向量表示。为了保证比赛的公平性，从中抽取20万用户的点击日志数据作为训练集，5万用户的点击日志数据作为测试集A，5万用户的点击日志数据作为测试集B。


## 4. 评价指标

- 根据sample.submit.csv， 我们最后提交的格式是针对每个用户，我们都会给出五篇文章的推荐结果，按照点击概率从前往后排序。而真实的每个用户最后一次点击的文章只会有一篇的真实答案，所以就看我们推荐的这五篇里面是否有命中真实答案的。
- 比如对于userA来说，我们的提交会是userA, article1, article2, article3, article4, article5。假如article1是用户的真实点击文章，也就代表article1命中，则s(userA,1)=1, s(userA,2-5)都是0；假如article2是用户的真实点击文章，也就代表article2命中，则s(userA,1)=0, s(userA, 2)=1, s(userA,3-5)都是0；即score(user)=命中第几条的倒数。如果没命中，则score(userA)=0
- 评价指标希望的是命中的结果越靠前，分数会越高



## 5. 赛题理解的过程

此次比赛的目标是根据用户历史浏览点击新闻的数据信息预测用户最后一次点击的新闻文章。
从这个目标上看，这和我们之前遇到的普通的结构化比赛不太一样，主要有两点：

- 目标上，要预测最后一次点击的新闻文章，并不是预测一个数或者预测数据分类
- 数据上，通过数据发现这不是典型的特征+标签，而是基于真实的业务场景下用户的点击日志

思考方向：结合目标，把该预测问题转成一个监督学习的问题(特征+标签)，然后才能进行ML，DL等建模预测。

问题：
- 1. 如何转化成一个监督学习问题
- 2. 转成一个什么样子的监督问题
- 3. 可以利用的特征有哪些
- 4. 可以尝试哪些模型 
- 5. 训练集和测试集怎么制作 

## 6. Baseline

使用的是协同过滤算法

In [1]:
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
import collections
from collections import defaultdict
warnings.filterwarnings('ignore')

In [2]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [3]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(sample_nums=10000):
    """
        训练集中采样一部分数据调试
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = reduce_mem(pd.read_csv(data_path + 'train_click_log.csv'))
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据分成线上和线下，
# 如果是为了获取线上提交结果应该将测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(offline=True):
    if offline:
        all_click = reduce_mem(pd.read_csv(data_path + 'train_click_log.csv'))
    else:
        trn_click = reduce_mem(pd.read_csv(data_path + 'train_click_log.csv'))
        tst_click = reduce_mem(pd.read_csv(data_path + 'testA_click_log.csv'))
        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

def make_item_time_pair(df):
    return list(zip(df['click_article_id'], df['click_timestamp']))
    
# 根据点击时间获取用户的点击文章序列 {user1: [(item1, time1), (item2, time2)..]...}
def get_user_item_time(click_df):
    click_df = click_df.sort_values('click_timestamp')
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(\
        lambda x: make_item_time_pair(x)).reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

# 获取近期点击最多的 TopK个文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

# itemCF的物品相似度计算
def item_cf_sim(df):
    """
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 在多路召回部分会加上关联规则的召回策略
    """
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open('./item_cf_i2i_sim.pkl', 'wb'))
    return i2i_sim_

In [4]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列{user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 [item1:score1, item2: score2...]
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id] # 注意，此时获取得到的是一个元组列表，[(30760, 1508211672520)]
    user_hist_items_ = {item_id for item_id, _ in user_hist_items}

    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue    
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]  
    return item_rank

In [5]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [6]:
# 读取数据
data_path = './data/source_data/'
all_click_df = get_all_click_df(offline=False)
user_item_time_dict = get_user_item_time(all_click_df)

i2i_sim = item_cf_sim(all_click_df) # 保存了item_cf_sim模型
item_topk_click = get_item_topk_click(all_click_df, k=50)

# 重新加载 item_cf_sim模型
# i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 与当前文章最相似的前k篇文章, 最后的召回文章数量

sim_item_topk, recall_item_num = 20, 10
user_recall_items_dict = collections.defaultdict(dict)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)


-- Mem. usage decreased to 23.34 Mb (69.4% reduction),time spend:0.00 min
-- Mem. usage decreased to 10.87 Mb (69.4% reduction),time spend:0.00 min


100%|██████████| 250000/250000 [1:11:04<00:00, 58.62it/s]


### 根据物品的协同过滤给每个用户推荐文章

In [8]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

# 从所有的召回数据中将测试集中的用户选出来
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='Luna_item_cf_baseline')

100%|██████████| 250000/250000 [00:04<00:00, 50245.74it/s]
